In [15]:
import pandas as pd
import re

In [16]:
data = pd.read_csv("L_STMaestroPrd.csv")

C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_13280\3658394942.py:1: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("L_STMaestroPrd.csv")


In [18]:
data.columns[data.isna().all()]
data = data.dropna(axis=1, how='all')
data["descrip1_descrip2"] = data["descrip1"].fillna('') + " " + data["descrip2"].fillna('')
data["descrip1_descrip2"]

0                                                      - -
1                                                      - -
2        (REPETIDO) SUPPORTAN 500ML CAJA X 15 SACHET X ...
3        BOLSAS PAPEL GRADO MEDICO 175X65X420MM - KIMS ...
4        BOLSAS PAPEL GRADO MEDICO 260X60X520MM - KIMS ...
                               ...                        
56769                           ZYVALIX 250 mg comp.x 120 
56770                       ZYVALIX PLUS 500 mg comp.x 60 
56771    ZYVOX IV 600 mg bolsas x 10x300ml CAJA X 10 BO...
56772                          ZYVOX ORAL 600 mg tab.x 10 
56773                                    ZZ TOP comp.x 30 
Name: descrip1_descrip2, Length: 56774, dtype: object

In [19]:
# 1. Eliminar espacios innecesarios.
# 2. Homogeneizar texto.

""" 
1.1. Eliminar Espacios Innecesarios
Los espacios extra pueden generar inconsistencias en la comparación de descripciones:

Problema: "Aspirina 10ml " y "Aspirina 10ml" se consideran diferentes.
Solución:
Quitar espacios al inicio y final de cada descripción.
Reducir múltiples espacios consecutivos entre palabras a un solo espacio.
Ejemplo:
Entrada: " Aspirina 10ml "
Salida: "Aspirina 10ml"

1.2. Homogeneizar Texto
La comparación debe ser insensible a mayúsculas/minúsculas:

Problema: "ASPIRINA" y "aspirina" se consideran diferentes.
Solución:
Convertir todo el texto a minúsculas o mayúsculas según convenga.
Ejemplo:
Entrada: "Aspirina Tableta"
Salida: "aspirina tableta"
"""

data['descrip1_descrip2'] = data['descrip1_descrip2'].str.strip().str.replace(r'\s+', ' ', regex=True).str.lower()
data['descrip1_descrip2']

0                                                      - -
1                                                      - -
2        (repetido) supportan 500ml caja x 15 sachet x ...
3        bolsas papel grado medico 175x65x420mm - kims ...
4        bolsas papel grado medico 260x60x520mm - kims ...
                               ...                        
56769                            zyvalix 250 mg comp.x 120
56770                        zyvalix plus 500 mg comp.x 60
56771    zyvox iv 600 mg bolsas x 10x300ml caja x 10 bo...
56772                           zyvox oral 600 mg tab.x 10
56773                                     zz top comp.x 30
Name: descrip1_descrip2, Length: 56774, dtype: object

In [20]:
# 3. Remover caracteres especiales.

"""
1.3. Eliminar Caracteres Especiales Innecesarios
Caracteres como #, %, *, pueden no aportar valor y generar ruido:

Problema: "Aspirina#10ml" no debería incluir #.
Solución:
Remover caracteres que no sean letras, números, espacios o separadores importantes como /, x.
Ejemplo:
Entrada: "Aspirina# 10ml % Blíster"
Salida: "Aspirina 10ml Blíster"
"""

# Remover caracteres no deseados, permitiendo letras, números, espacios, 'x', '/' y '%'
data['descrip1_descrip2'] = data['descrip1_descrip2'].str.replace(r'[^a-zA-Z0-9\s%]', '', regex=True)


data["descrip1_descrip2"]

0                                                         
1                                                         
2        repetido supportan 500ml caja x 15 sachet x 50...
3        bolsas papel grado medico 175x65x420mm  kims b...
4        bolsas papel grado medico 260x60x520mm  kims p...
                               ...                        
56769                             zyvalix 250 mg compx 120
56770                         zyvalix plus 500 mg compx 60
56771    zyvox iv 600 mg bolsas x 10x300ml caja x 10 bo...
56772                            zyvox oral 600 mg tabx 10
56773                                      zz top compx 30
Name: descrip1_descrip2, Length: 56774, dtype: object

In [34]:
def estandarizar_descripcion(texto):
    # 1. Separar números de letras (ej: 420mm -> 420 mm)
    texto = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', texto)
    texto = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', texto)
    
    # 2. Agregar espacios alrededor de la 'x' usada como separador (ej: 10x10 -> 10 x 10)
    texto = re.sub(r'(?<=\d)x(?=\d)', r' x ', texto)
    
    # 3. Normalizar términos específicos combinados (ej: tabx -> tab x)
    texto = re.sub(r'\b(tabx)\b', r'tab x', texto, flags=re.IGNORECASE)
    texto = re.sub(r'\b(compx)\b', r'comp x', texto, flags=re.IGNORECASE)
    
    # 4. Reemplazar separadores como guiones o barras con espacios
    texto = re.sub(r'[-/]', ' ', texto)
    
    # 5. Remover espacios redundantes
    texto = re.sub(r'\s+', ' ', texto).strip()
    
    return texto

# Aplicar la función al DataFrame
data['descrip1_descrip2'] = data['descrip1_descrip2'].apply(estandarizar_descripcion)

data["descrip1_descrip2"].iloc[100:150]



100                                    aha crx 75
101            a 4 body lotion durazno locx 220 g
102            a 4 body lotion manzana locx 220 g
103               a 4 body lotion meln locx 220 g
104                        a 4 calendula crx 60 g
105              abacavir elea 300 mg comprecx 60
106    abacavir lamivudina trb pharma comprecx 30
107                      abaglin 100 mg capsx 100
108                       abaglin 100 mg capsx 30
109                       abaglin 100 mg capsx 50
110                      abaglin 300 mg capsx 100
111                       abaglin 300 mg capsx 30
112                       abaglin 300 mg capsx 50
113                       abaglin 300 mg capsx 60
114                      abaglin 400 mg capsx 100
115                       abaglin 400 mg capsx 30
116                       abaglin 400 mg capsx 50
117                       abaglin 400 mg capsx 60
118               abanta emulslimpiadora x 250 ml
119               abanta emulslimpiadora x 500 ml
